# **Impact Assessment**

### Import processing and prepping libraries

In [1]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

import psycopg2
import pymysql

from datetime import timedelta

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Load the Data

In [4]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'ubuntuuser'
password = '97cD8$pt@'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM bloomlive.scoring_results" 
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()
    
if __name__ == "__main__":
    get_query_results_postgres()  

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
df_scoring_summaries = get_query_results_postgres()

df_scoring_summaries.head(3)

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,surrogate_id,store_number,national_id,final_21_limit,final_7_limit,final_1_limit,idm_recommendation,limit_factor_21,limit_factor_7,limit_factor_1,model_version,blacklist_flag,created_at,record_added_to_warehouse_on_timestamp,limit_reason
0,187914,822369,22128407,0.000,0.000,0.000,Approve,0.500,0.170,0.170,"2022-001[2022-01-05, 2022-02-01]",1,2022-02-01,2022-02-01 10:41:30:077609,None
1,187915,582756,22935173,0.000,0.000,0.000,Approve,0.500,0.170,0.170,"2022-001[2022-01-05, 2022-02-01]",1,2022-02-01,2022-02-01 10:41:30:077609,None
2,29,None,13489593,60000.000,50900.000,9000.000,Approve,0.500,0.170,0.030,"2021-001[2021-10-18, 2021-11-02]",1,2021-11-02,2022-01-10 12:22:15.174327,None


### Clean the Data

In [6]:
# creating a copy of the dataframe
df_scoring_summaries_copy = df_scoring_summaries.copy()

In [7]:
# checking the columns in the dataframe
df_scoring_summaries.columns

Index(['surrogate_id', 'store_number', 'national_id', 'final_21_limit', 'final_7_limit', 'final_1_limit', 'idm_recommendation', 'limit_factor_21', 'limit_factor_7', 'limit_factor_1', 'model_version', 'blacklist_flag', 'created_at', 'record_added_to_warehouse_on_timestamp', 'limit_reason'], dtype='object')

In [8]:
df_scoring_summaries['model_version'].unique()

array(['2022-001[2022-01-05, 2022-02-01]',
       '2021-001[2021-10-18, 2021-11-02]',
       '2021-001[2021-10-18, 2021-12-08]',
       '2021-001[2021-10-18, 2021-12-21]',
       '2022-001[2022-01-05, 2022-01-05]',
       '2022-001[2022-01-05, 2022-01-10]',
       '2022-001[2022-01-05, 2022-01-17]',
       '2022-001[2022-01-05, 2022-01-24]',
       '2022-001[2022-01-05, 2022-02-07]',
       '2022-003[2022-02-27, 2022-03-10]',
       '2022-001[2022-01-05, 2022-02-14]',
       '2022-002[2022-02-24, 2022-02-27]',
       '2022-003[2022-02-27, 2022-05-05]',
       '2022-003[2022-02-27, 2022-03-03]',
       '2022-004[2022-05-14, 2022-05-14]',
       '2022-003[2022-02-27, 2022-05-12]',
       '2022-003[2022-02-27, 2022-03-24]',
       '2022-004[2022-05-14, 2022-05-23]',
       '2022-003[2022-02-27, 2022-04-07]',
       '2022-003[2022-02-27, 2022-04-21]',
       '2022-003[2022-02-27, 2022-03-30]',
       '2022-004[2022-05-14, 2022-05-21]',
       '2022-003[2022-02-27, 2022-04-13]',
       '202

In [9]:
# filtering to get only march refreshes
df_scoring_summaries = df_scoring_summaries[(df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-03]') | (df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-10]') | (df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-24]') | (df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-30]')]

In [10]:
# checking the number of unique refreshes
df_scoring_summaries['model_version'].nunique()

4

In [11]:
# checking the unique refreshes
df_scoring_summaries['model_version'].unique()

array(['2022-003[2022-02-27, 2022-03-10]',
       '2022-003[2022-02-27, 2022-03-03]',
       '2022-003[2022-02-27, 2022-03-24]',
       '2022-003[2022-02-27, 2022-03-30]'], dtype=object)

In [12]:
# checking info of the dataframe
# detecting null values in days_past_due column
df_scoring_summaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143830 entries, 196589 to 717903
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   surrogate_id                            143830 non-null  int64  
 1   store_number                            143830 non-null  object 
 2   national_id                             143830 non-null  object 
 3   final_21_limit                          143830 non-null  float64
 4   final_7_limit                           143830 non-null  float64
 5   final_1_limit                           143830 non-null  float64
 6   idm_recommendation                      143830 non-null  object 
 7   limit_factor_21                         143830 non-null  float64
 8   limit_factor_7                          143830 non-null  float64
 9   limit_factor_1                          143830 non-null  float64
 10  model_version                          

In [13]:
current_refresh = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220726_multiple_products_9.xlsx")
current_refresh = current_refresh[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]
current_refresh = current_refresh.rename(columns={'final_21_limit':'new_final_21_limit', 'final_7_limit':'new_final_7_limit', 'final_1_limit':'new_final_1_limit'})

current_refresh.head()

,store_number,new_final_21_limit,new_final_7_limit,new_final_1_limit
0,822369,0,0,0
1,165978,0,23000,23000
2,786152,0,0,0
3,745142,0,0,0
4,978161,0,0,0


In [14]:
current_refresh[['new_final_21_limit', 'new_final_7_limit', 'new_final_1_limit']].sum()

new_final_21_limit    322162400
new_final_7_limit     473361100
new_final_1_limit     475361400
dtype: int64

In [15]:
current_refresh['store_number'] = current_refresh['store_number'].astype(str)

### 1. Checking impact on change in limits allocated for each March refresh

In [16]:
# defining a function for the limit caps
def calc_limit_difference_bands(df):
    limits_difference = df['limits_difference']
    
    if limits_difference < 0:
        return 'Decrease in Limits'
    elif limits_difference == 0:
        return 'No Change in Limits'
    elif limits_difference > 0:
        return 'Increase in Limits'

#### First March Refresh

In [17]:
refresh_one = df_scoring_summaries[df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-03]']
refresh_one = refresh_one[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]

In [18]:
refresh_one.shape

(34073, 4)

##### 21 Day Loans

In [19]:
old_21_day_limits = refresh_one.groupby(['store_number'], as_index=False)['final_21_limit'].sum()
new_21_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_21_limit'].sum()

refresh_one_21_day_limits = pd.merge(old_21_day_limits, new_21_day_limits, on = "store_number", how = 'left')

refresh_one_21_day_limits = refresh_one_21_day_limits.fillna(0)

refresh_one_21_day_limits['limits_difference'] = refresh_one_21_day_limits['new_final_21_limit'] - refresh_one_21_day_limits['final_21_limit']

refresh_one_21_day_limits.head(2)

,store_number,final_21_limit,new_final_21_limit,limits_difference
0,103914,0.000,0.000,0.000
1,105295,200000.000,0.000,-200000.000


In [20]:
refresh_one_21_day_limits['limits_difference_bands'] = refresh_one_21_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_one_21_day_limits.head(2)

,store_number,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands
0,103914,0.000,0.000,0.000,No Change in Limits
1,105295,200000.000,0.000,-200000.000,Decrease in Limits


In [21]:
refresh_one_21_day_limits.shape

(34073, 5)

In [22]:
refresh_one_21_day_limits_difference_bands = refresh_one_21_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_one_21_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-86442100.000
1,Increase in Limits,195308100.000
2,No Change in Limits,0.000


In [23]:
total_initial_21_day_limits = refresh_one_21_day_limits['final_21_limit'].sum()

total_initial_21_day_limits

160387600.0

In [24]:
total_new_limits_21_day_limits = refresh_one_21_day_limits['new_final_21_limit'].sum()

total_new_limits_21_day_limits

269253600.0

##### 7 Day Loans

In [25]:
old_7_day_limits = refresh_one.groupby(['store_number'], as_index=False)['final_7_limit'].sum()
new_7_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_7_limit'].sum()

refresh_one_7_day_limits = pd.merge(old_7_day_limits, new_7_day_limits, on = "store_number", how = 'left')

refresh_one_7_day_limits = refresh_one_7_day_limits.fillna(0)

refresh_one_7_day_limits['limits_difference'] = refresh_one_7_day_limits['new_final_7_limit'] - refresh_one_7_day_limits['final_7_limit']

refresh_one_7_day_limits['limits_difference_bands'] = refresh_one_7_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_one_7_day_limits_difference_bands = refresh_one_7_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_one_7_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-245610200.000
1,Increase in Limits,127543100.000
2,No Change in Limits,0.000


In [26]:
total_initial_7_day_limits = refresh_one_7_day_limits['final_7_limit'].sum()

total_initial_7_day_limits

478784600.0

In [27]:
total_new_limits_7_day_limits = refresh_one_7_day_limits['new_final_7_limit'].sum()

total_new_limits_7_day_limits

360717500.0

##### 1 Day Loans

In [28]:
old_1_day_limits = refresh_one.groupby(['store_number'], as_index=False)['final_1_limit'].sum()
new_1_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_1_limit'].sum()

refresh_one_1_day_limits = pd.merge(old_1_day_limits, new_1_day_limits, on = "store_number", how = 'left')

refresh_one_1_day_limits = refresh_one_1_day_limits.fillna(0)

refresh_one_1_day_limits['limits_difference'] = refresh_one_1_day_limits['new_final_1_limit'] - refresh_one_1_day_limits['final_1_limit']

refresh_one_1_day_limits['limits_difference_bands'] = refresh_one_1_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_one_1_day_limits_difference_bands = refresh_one_1_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_one_1_day_limits_difference_bands



,limits_difference_bands,limits_difference
0,Decrease in Limits,-246026000.000
1,Increase in Limits,127393000.000
2,No Change in Limits,0.000


In [29]:
total_initial_1_day_limits = refresh_one_1_day_limits['final_1_limit'].sum()

total_initial_1_day_limits

480061400.0

In [30]:
total_new_limits_1_day_limits = refresh_one_1_day_limits['new_final_1_limit'].sum()

total_new_limits_1_day_limits

361428400.0

#### Second March Refresh

In [31]:
refresh_two = df_scoring_summaries[df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-10]']
refresh_two = refresh_two[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]

In [32]:
refresh_two.shape

(34895, 4)

##### 21 Day Loans

In [33]:
old_21_day_limits = refresh_two.groupby(['store_number'], as_index=False)['final_21_limit'].sum()
new_21_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_21_limit'].sum()

refresh_two_21_day_limits = pd.merge(old_21_day_limits, new_21_day_limits, on = "store_number", how = 'left')

refresh_two_21_day_limits = refresh_two_21_day_limits.fillna(0)

refresh_two_21_day_limits['limits_difference'] = refresh_two_21_day_limits['new_final_21_limit'] - refresh_two_21_day_limits['final_21_limit']

refresh_two_21_day_limits['limits_difference_bands'] = refresh_two_21_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_two_21_day_limits_difference_bands = refresh_two_21_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_two_21_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-150227900.000
1,Increase in Limits,158469600.000
2,No Change in Limits,0.000


In [34]:
total_initial_21_day_limits = refresh_two_21_day_limits['final_21_limit'].sum()

total_initial_21_day_limits

266385400.0

In [35]:
total_new_21_day_limits = refresh_two_21_day_limits['new_final_21_limit'].sum()

total_new_21_day_limits

274627100.0

##### 7 Day Loans

In [36]:
old_7_day_limits = refresh_two.groupby(['store_number'], as_index=False)['final_7_limit'].sum()
new_7_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_7_limit'].sum()

refresh_two_7_day_limits = pd.merge(old_7_day_limits, new_7_day_limits, on = "store_number", how = 'left')

refresh_two_7_day_limits = refresh_two_7_day_limits.fillna(0)

refresh_two_7_day_limits['limits_difference'] = refresh_two_7_day_limits['new_final_7_limit'] - refresh_two_7_day_limits['final_7_limit']

refresh_two_7_day_limits['limits_difference_bands'] = refresh_two_7_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_two_7_day_limits_difference_bands = refresh_two_7_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_two_7_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-254997900.000
1,Increase in Limits,126768800.000
2,No Change in Limits,0.000


In [37]:
refresh_two_7_day_limits.shape

(34895, 5)

In [38]:
total_initial_7_day_limits = refresh_two_7_day_limits['final_7_limit'].sum()

total_initial_7_day_limits

495057700.0

In [39]:
total_new_7_day_limits = refresh_two_7_day_limits['new_final_7_limit'].sum()

total_new_7_day_limits

366828600.0

##### 1 Day Loans

In [40]:
old_1_day_limits = refresh_two.groupby(['store_number'], as_index=False)['final_1_limit'].sum()
new_1_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_1_limit'].sum()

refresh_two_1_day_limits = pd.merge(old_1_day_limits, new_1_day_limits, on = "store_number", how = 'left')

refresh_two_1_day_limits = refresh_two_1_day_limits.fillna(0)

refresh_two_1_day_limits['limits_difference'] = refresh_two_1_day_limits['new_final_1_limit'] - refresh_two_1_day_limits['final_1_limit']

refresh_two_1_day_limits['limits_difference_bands'] = refresh_two_1_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_two_1_day_limits_difference_bands = refresh_two_1_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_two_1_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-255359500.000
1,Increase in Limits,126642400.000
2,No Change in Limits,0.000


In [41]:
total_initial_1_day_limits = refresh_two_1_day_limits['final_1_limit'].sum()

total_initial_1_day_limits

496284700.0

In [42]:
total_new_1_day_limits = refresh_two_1_day_limits['new_final_1_limit'].sum()

total_new_1_day_limits

367567600.0

#### Third March Refresh

In [43]:
refresh_three = df_scoring_summaries[df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-24]']
refresh_three = refresh_three[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]

In [44]:
refresh_three.shape

(37198, 4)

##### 21 Day Loans

In [45]:
old_21_day_limits = refresh_three.groupby(['store_number'], as_index=False)['final_21_limit'].sum()
new_21_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_21_limit'].sum()

refresh_three_21_day_limits = pd.merge(old_21_day_limits, new_21_day_limits, on = "store_number", how = 'left')

refresh_three_21_day_limits = refresh_three_21_day_limits.fillna(0)

refresh_three_21_day_limits['limits_difference'] = refresh_three_21_day_limits['new_final_21_limit'] - refresh_three_21_day_limits['final_21_limit']

refresh_three_21_day_limits['limits_difference_bands'] = refresh_three_21_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_three_21_day_limits_difference_bands = refresh_three_21_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_three_21_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-181857000.000
1,Increase in Limits,146281100.000
2,No Change in Limits,0.000


In [46]:
total_initial_21_day_limits = refresh_three_21_day_limits['final_21_limit'].sum()

total_initial_21_day_limits

318771600.0

In [47]:
total_new_21_day_limits = refresh_three_21_day_limits['new_final_21_limit'].sum()

total_new_21_day_limits

283195700.0

##### 7 Day Loans

In [48]:
old_7_day_limits = refresh_three.groupby(['store_number'], as_index=False)['final_7_limit'].sum()
new_7_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_7_limit'].sum()

refresh_three_7_day_limits = pd.merge(old_7_day_limits, new_7_day_limits, on = "store_number", how = 'left')

refresh_three_7_day_limits = refresh_three_7_day_limits.fillna(0)

refresh_three_7_day_limits['limits_difference'] = refresh_three_7_day_limits['new_final_7_limit'] - refresh_three_7_day_limits['final_7_limit']

refresh_three_7_day_limits['limits_difference_bands'] = refresh_three_7_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_three_7_day_limits_difference_bands = refresh_three_7_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_three_7_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-270048800.000
1,Increase in Limits,127571700.000
2,No Change in Limits,0.000


In [49]:
total_initial_7_day_limits = refresh_three_7_day_limits['final_7_limit'].sum()

total_initial_7_day_limits

520519400.0

In [50]:
total_new_7_day_limits = refresh_three_7_day_limits['new_final_7_limit'].sum()

total_new_7_day_limits

378042300.0

##### 1 Day Loans

In [51]:
old_1_day_limits = refresh_three.groupby(['store_number'], as_index=False)['final_1_limit'].sum()
new_1_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_1_limit'].sum()

refresh_three_1_day_limits = pd.merge(old_1_day_limits, new_1_day_limits, on = "store_number", how = 'left')

refresh_three_1_day_limits = refresh_three_1_day_limits.fillna(0)

refresh_three_1_day_limits['limits_difference'] = refresh_three_1_day_limits['new_final_1_limit'] - refresh_three_1_day_limits['final_1_limit']

refresh_three_1_day_limits['limits_difference_bands'] = refresh_three_1_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_three_1_day_limits_difference_bands = refresh_three_1_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_three_1_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-270445500.000
1,Increase in Limits,127428900.000
2,No Change in Limits,0.000


In [52]:
total_initial_1_day_limits = refresh_three_1_day_limits['final_1_limit'].sum()

total_initial_1_day_limits

521855500.0

In [53]:
total_new_1_day_limits = refresh_three_1_day_limits['new_final_1_limit'].sum()

total_new_1_day_limits

378838900.0

#### Fourth March Refresh

In [54]:
refresh_four = df_scoring_summaries[df_scoring_summaries['model_version'] == '2022-003[2022-02-27, 2022-03-30]']
refresh_four = refresh_four[['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit']]

In [55]:
refresh_four.shape

(37664, 4)

##### 21 Day Loans

In [56]:
old_21_day_limits = refresh_four.groupby(['store_number'], as_index=False)['final_21_limit'].sum()
new_21_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_21_limit'].sum()

refresh_four_21_day_limits = pd.merge(old_21_day_limits, new_21_day_limits, on = "store_number", how = 'left')

refresh_four_21_day_limits = refresh_four_21_day_limits.fillna(0)

refresh_four_21_day_limits['limits_difference'] = refresh_four_21_day_limits['new_final_21_limit'] - refresh_four_21_day_limits['final_21_limit']

refresh_four_21_day_limits['limits_difference_bands'] = refresh_four_21_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_four_21_day_limits_difference_bands = refresh_four_21_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_four_21_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-182717300.000
1,Increase in Limits,145282900.000
2,No Change in Limits,0.000


In [57]:
total_initial_21_day_limits = refresh_four_21_day_limits['final_21_limit'].sum()

total_initial_21_day_limits

323795900.0

In [58]:
total_new_21_day_limits = refresh_four_21_day_limits['new_final_21_limit'].sum()

total_new_21_day_limits

286361500.0

##### 7 Day Loans

In [59]:
old_7_day_limits = refresh_four.groupby(['store_number'], as_index=False)['final_7_limit'].sum()
new_7_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_7_limit'].sum()

refresh_four_7_day_limits = pd.merge(old_7_day_limits, new_7_day_limits, on = "store_number", how = 'left')

refresh_four_7_day_limits = refresh_four_7_day_limits.fillna(0)

refresh_four_7_day_limits['limits_difference'] = refresh_four_7_day_limits['new_final_7_limit'] - refresh_four_7_day_limits['final_7_limit']

refresh_four_7_day_limits['limits_difference_bands'] = refresh_four_7_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_four_7_day_limits_difference_bands = refresh_four_7_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_four_7_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-268036500.000
1,Increase in Limits,125755700.000
2,No Change in Limits,0.000


In [60]:
total_initial_7_day_limits = refresh_four_7_day_limits['final_7_limit'].sum()

total_initial_7_day_limits

524452800.0

In [61]:
total_new_7_day_limits = refresh_four_7_day_limits['new_final_7_limit'].sum()

total_new_7_day_limits

382172000.0

##### 1 Day Loans

In [62]:
old_1_day_limits = refresh_four.groupby(['store_number'], as_index=False)['final_1_limit'].sum()
new_1_day_limits = current_refresh.groupby(['store_number'], as_index=False)['new_final_1_limit'].sum()

refresh_four_1_day_limits = pd.merge(old_1_day_limits, new_1_day_limits, on = "store_number", how = 'left')

refresh_four_1_day_limits = refresh_four_1_day_limits.fillna(0)

refresh_four_1_day_limits['limits_difference'] = refresh_four_1_day_limits['new_final_1_limit'] - refresh_four_1_day_limits['final_1_limit']

refresh_four_1_day_limits['limits_difference_bands'] = refresh_four_1_day_limits.apply(lambda x: calc_limit_difference_bands(x), axis = 1)

refresh_four_1_day_limits_difference_bands = refresh_four_1_day_limits.groupby(['limits_difference_bands'], as_index=False)['limits_difference'].sum()

refresh_four_1_day_limits_difference_bands

,limits_difference_bands,limits_difference
0,Decrease in Limits,-268368600.000
1,Increase in Limits,125614900.000
2,No Change in Limits,0.000


In [63]:
total_initial_1_day_limits = refresh_four_1_day_limits['final_1_limit'].sum()

total_initial_1_day_limits

525755600.0

In [64]:
total_new_1_day_limits = refresh_four_1_day_limits['new_final_1_limit'].sum()

total_new_1_day_limits

383001900.0

### 2. Check impact on subsequent loan take-ups

In [65]:
host = '157.245.248.249'
port = int(5432)
dbname = 'ubuntu'
user = 'ubuntuuser'
password = '97cD8$pt@'


def get_query_results_postgres():
    with psycopg2.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = 'SELECT * FROM bloomlive.loans_fact_table_summary_view where principal_disbursed > 0'
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()
    
if __name__ == "__main__":
    get_query_results_postgres()  

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [66]:
df_loans = get_query_results_postgres()

df_loans.head(3)

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90
0,3056468,147966,2.000,172336,90397,254722343894,66,2,NaN,NaN,600,000147966,Individual Loan,mifos-standard-strategy,7,Days,7,6200.000,6200.000,6200.000,6200.000,0.000,0.000,161.210,161.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6361.210,6361.210,161.210,161.210,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7124626,3.000,None,QG62YDF1WW,6200.000,135.571,47,3.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110200000001054272,2022-07-06 11:33:22,448.000,6200.000,6200.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14
1,3056471,147980,2.000,115249,27265,254727603605,66,2,NaN,NaN,600,000147980,Individual Loan,mifos-standard-strategy,7,Days,7,37000.000,37000.000,37000.000,37000.000,0.000,0.000,962.010,962.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37962.010,37962.010,962.010,962.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,962364,11.000,None,QG63YE5V1N,37000.000,135.571,47,10.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110500000001052255,2022-07-06 11:43:30,427.000,37000.000,37000.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14
2,3056544,148223,2.000,96155,7855,254703160599,66,2,NaN,NaN,600,000148223,Individual Loan,mifos-standard-strategy,7,Days,7,34000.000,34000.000,34000.000,34000.000,0.000,0.000,884.010,884.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,34884.010,34884.010,884.010,884.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,N

In [67]:
#changing the dates column to datetime datatype
df_loans['disbursed_on_date'] = pd.to_datetime(df_loans['disbursed_on_date'])
df_loans['expected_matured_on_date'] = pd.to_datetime(df_loans['expected_matured_on_date'])

In [68]:
df_loans.head()

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90
0,3056468,147966,2.000,172336,90397,254722343894,66,2,NaN,NaN,600,000147966,Individual Loan,mifos-standard-strategy,7,Days,7,6200.000,6200.000,6200.000,6200.000,0.000,0.000,161.210,161.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6361.210,6361.210,161.210,161.210,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7124626,3.000,None,QG62YDF1WW,6200.000,135.571,47,3.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110200000001054272,2022-07-06 11:33:22,448.000,6200.000,6200.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14
1,3056471,147980,2.000,115249,27265,254727603605,66,2,NaN,NaN,600,000147980,Individual Loan,mifos-standard-strategy,7,Days,7,37000.000,37000.000,37000.000,37000.000,0.000,0.000,962.010,962.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37962.010,37962.010,962.010,962.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,962364,11.000,None,QG63YE5V1N,37000.000,135.571,47,10.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110500000001052255,2022-07-06 11:43:30,427.000,37000.000,37000.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14
2,3056544,148223,2.000,96155,7855,254703160599,66,2,NaN,NaN,600,000148223,Individual Loan,mifos-standard-strategy,7,Days,7,34000.000,34000.000,34000.000,34000.000,0.000,0.000,884.010,884.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,34884.010,34884.010,884.010,884.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,N

In [69]:
df_loans = df_loans[df_loans['bloom_version'] == 2]

df_loans = df_loans[(df_loans['loan_status'] == 300) | (df_loans['loan_status'] == 600)| (df_loans['loan_status'] == 700)]

df_loans.shape

(175784, 106)

In [70]:
#create due_date fixed column to ensure due date tallies with the term frequency for each loan

df_loans["due_date_fixed"] = df_loans["disbursed_on_date"] + pd.to_timedelta(df_loans["term_frequency"], unit='d')

df_loans['due_date_fixed'] = pd.to_datetime(df_loans['due_date_fixed'])

df_loans.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,due_date_fixed
0,3056468,147966,2.000,172336,90397,254722343894,66,2,NaN,NaN,600,000147966,Individual Loan,mifos-standard-strategy,7,Days,7,6200.000,6200.000,6200.000,6200.000,0.000,0.000,161.210,161.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6361.210,6361.210,161.210,161.210,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7124626,3.000,None,QG62YDF1WW,6200.000,135.571,47,3.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110200000001054272,2022-07-06 11:33:22,448.000,6200.000,6200.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13
1,3056471,147980,2.000,115249,27265,254727603605,66,2,NaN,NaN,600,000147980,Individual Loan,mifos-standard-strategy,7,Days,7,37000.000,37000.000,37000.000,37000.000,0.000,0.000,962.010,962.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37962.010,37962.010,962.010,962.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,962364,11.000,None,QG63YE5V1N,37000.000,135.571,47,10.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110500000001052255,2022-07-06 11:43:30,427.000,37000.000,37000.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13


In [71]:
#converting the current time to datetime format
curr_time = pd.to_datetime('today')

In [72]:
def calc_end_rollover_dates(df):
    tenure = df['repay_every']
    expected_matured_on_date = df['expected_matured_on_date']
    
    if tenure == 21:
        res = expected_matured_on_date + pd.to_timedelta(5, unit='d')
    elif tenure == 7:
        res = expected_matured_on_date + pd.to_timedelta(3, unit='d')
    elif tenure == 1:
        res = expected_matured_on_date + pd.to_timedelta(1, unit='d')
        
    return res

In [73]:
df_loans['end_rollover_dates'] = df_loans.apply(lambda x: calc_end_rollover_dates(x), axis = 1)

df_loans.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,due_date_fixed,end_rollover_dates
0,3056468,147966,2.000,172336,90397,254722343894,66,2,NaN,NaN,600,000147966,Individual Loan,mifos-standard-strategy,7,Days,7,6200.000,6200.000,6200.000,6200.000,0.000,0.000,161.210,161.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6361.210,6361.210,161.210,161.210,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7124626,3.000,None,QG62YDF1WW,6200.000,135.571,47,3.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110200000001054272,2022-07-06 11:33:22,448.000,6200.000,6200.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13,2022-07-16
1,3056471,147980,2.000,115249,27265,254727603605,66,2,NaN,NaN,600,000147980,Individual Loan,mifos-standard-strategy,7,Days,7,37000.000,37000.000,37000.000,37000.000,0.000,0.000,962.010,962.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37962.010,37962.010,962.010,962.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,962364,11.000,None,QG63YE5V1N,37000.000,135.571,47,10.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110500000001052255,2022-07-06 11:43:30,427.000,37000.000,37000.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13,2022-07-16


In [74]:
#create days past due date column to ensure due date tallies with the term frequency for each loan

df_loans["dpd_90_date"] = df_loans["end_rollover_dates"] + pd.to_timedelta(90, unit='d')

df_loans['dpd_90_date'] = pd.to_datetime(df_loans['dpd_90_date'])

df_loans.head(2)

,loan_surrogate_id,loan_mifos_id,bloom_version,client_surrogate_id,client_mifos_id,client_mobile_number,product_surrogate_id,product_mifos_id,loan_officer_surrogate_id,loan_officer_mifos_id,loan_status,loan_account_number,loan_type,transaction_strategy,term_frequency,term_period_frequency,repay_every,principal_amount_proposed,principal_amount,principal_disbursed,principal_repaid,principal_written_off,principal_outstanding,interest_charged,interest_repaid,interest_waived,interest_written_off,interest_outstanding,fee_charges_charged,fee_charges_repaid,fee_charges_waived,fee_charges_written_off,fee_charges_outstanding,penalty_charges_charged,penalty_charges_repaid,penalty_charges_waived,penalty_charges_written_off,penalty_charges_outstanding,total_expected_repayment,total_repayment,total_expected_cost_of_loan,total_cost_of_loan,total_charges_due_at_disbursement,total_waived,total_written_off,total_outstanding,total_overpaid,total_recovered,guarantee_amount,arrears_tolerance_amount,interest_rate_differential,fixed_emi_amount,max_outstanding_loan_balance,accrued_till,submitted_on_date,approved_on_date,expected_disbursed_on_date,disbursed_on_date,expected_matured_on_date,matured_on_date,expected_first_repayment_on_date,closed_on_date,rescheduled_on_date,rejected_on_date,withdrawn_on_date,written_off_on_date,interest_calculated_from_date,interest_recalculated_on,allow_partial_period_interest_calculation,is_floating_interest_rate,sync_disbursement_with_meeting,interest_recalculation_enabled,create_standing_instruction_at_disbursement,is_npa,is_topup,external_id,store_number,counter,model_version,receipt_number,approved_principal,annual_nominal_interest_rate,loan_status_surrogate_id,product_counter,grace_on_arrears_ageing,loan_version,submitted_on_user_id,closed_on_user_id,rescheduled_on_user_id,withdrawn_on_user_id,disbursed_on_user_id,approved_on_user_id,rejected_on_user_id,record_created_on_date,cntrct_id,application_timestamp,src_crdt_score,src_assgnd_crdt_lmt,src_used_crdt_lmit,src_avail_crdt_lmit,credit_limit_balance,src_id_prdct,end_rollvr_dt,dpd_30,dpd_d60,dpd_d90,due_date_fixed,end_rollover_dates,dpd_90_date
0,3056468,147966,2.000,172336,90397,254722343894,66,2,NaN,NaN,600,000147966,Individual Loan,mifos-standard-strategy,7,Days,7,6200.000,6200.000,6200.000,6200.000,0.000,0.000,161.210,161.210,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6361.210,6361.210,161.210,161.210,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,7124626,3.000,None,QG62YDF1WW,6200.000,135.571,47,3.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110200000001054272,2022-07-06 11:33:22,448.000,6200.000,6200.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13,2022-07-16,2022-10-14
1,3056471,147980,2.000,115249,27265,254727603605,66,2,NaN,NaN,600,000147980,Individual Loan,mifos-standard-strategy,7,Days,7,37000.000,37000.000,37000.000,37000.000,0.000,0.000,962.010,962.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37962.010,37962.010,962.010,962.010,0.000,0.000,0.000,0.000,NaN,None,None,None,None,None,None,None,2022-07-06,2022-07-06,2022-07-06,2022-07-06,2022-07-13,2022-07-13,None,2022-07-13,None,None,None,None,None,None,False,None,None,False,None,False,False,None,962364,11.000,None,QG63YE5V1N,37000.000,135.571,47,10.000,NaN,6,1,NaN,None,None,1,1.000,NaN,2022-07-13 15:21:33.792215,110500000001052255,2022-07-06 11:43:30,427.000,37000.000,37000.000,0.000,None,10002,2022-07-16,2022-08-15,2022-09-14,2022-10-14,2022-07-13,2022-07-16,2022-10-14


In [75]:
df_loans = df_loans[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'disbursed_on_date', 'expected_matured_on_date', 'total_expected_repayment','due_date_fixed', 'end_rollover_dates', 'dpd_90_date']]

df_loans.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,due_date_fixed,end_rollover_dates,dpd_90_date
0,147966,7124626,7,6200.000,2022-07-06,2022-07-13,6361.210,2022-07-13,2022-07-16,2022-10-14
1,147980,962364,7,37000.000,2022-07-06,2022-07-13,37962.010,2022-07-13,2022-07-16,2022-10-14


In [76]:
host = '167.71.47.216'
port = int(3306)
user = 'tableau_reports'
password = 'change@123'
dbname = 'mifostenant-safaricom'

def get_query_results_mysql():
    with pymysql.connect(host = host,
                         port = port,
                         database = dbname,
                         user = user,
                         password = password) as conn:
        sql = "SELECT * FROM m_loan_transaction"
        df = pd.read_sql(sql, conn)
        
    return df

    conn.close()

In [77]:
df_transactions = get_query_results_mysql()

df_transactions.head(2)

/opt/tljh/user/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,loan_id,office_id,payment_detail_id,is_reversed,external_id,transaction_type_enum,transaction_date,amount,principal_portion_derived,interest_portion_derived,fee_charges_portion_derived,penalty_charges_portion_derived,overpayment_portion_derived,unrecognized_income_portion,outstanding_loan_balance_derived,submitted_on_date,manually_adjusted_or_reversed,created_date,appuser_id
0,13999,3697,1,7245.000,0,None,1,2021-11-05,150.000,NaN,NaN,NaN,NaN,NaN,None,150.000,2021-12-27,0,2021-12-27 12:26:34,1
1,14000,3697,1,NaN,0,None,10,2021-11-05,5.090,NaN,5.090,NaN,NaN,NaN,None,NaN,2021-12-27,0,2021-12-27 12:26:34,1


In [78]:
# filtering df to only contain repayments
df_transactions = df_transactions[(df_transactions['transaction_type_enum'] == 2) & (df_transactions['is_reversed'] == 0)]

df_transactions.shape

(325018, 20)

In [79]:
#renaming loan_id to loan_mifos_id
df_transactions.rename(columns = {'loan_id':'loan_mifos_id'}, inplace = True)

In [80]:
df_transactions['transaction_date'] = pd.to_datetime(df_transactions['transaction_date'], errors='coerce')

In [84]:
max_trans_date = df_transactions.groupby(['loan_mifos_id'], as_index=False)['transaction_date'].max()
max_trans_date.rename(columns = {'transaction_date':'max_transaction_date'}, inplace = True)

max_trans_date.head()

,loan_mifos_id,max_transaction_date
0,3697,2021-11-05
1,3698,2021-11-05
2,3700,2021-11-26
3,3701,2021-11-26
4,3702,2021-11-12


In [85]:
df_transactions = pd.merge(df_transactions, max_trans_date, on = 'loan_mifos_id', how = 'left')
df_transactions = df_transactions[['loan_mifos_id', 'transaction_date', 'amount', 'max_transaction_date']]

df_transactions.head(2)

,loan_mifos_id,transaction_date,amount,max_transaction_date
0,3698,2021-11-05,30.000,2021-11-05
1,3703,2021-11-05,111.000,2021-11-05


In [86]:
df2 = pd.merge(df_loans, df_transactions, on = 'loan_mifos_id', how = 'left')

df2.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,due_date_fixed,end_rollover_dates,dpd_90_date,transaction_date,amount,max_transaction_date
0,147966,7124626,7,6200.000,2022-07-06,2022-07-13,6361.210,2022-07-13,2022-07-16,2022-10-14,2022-07-12,6361.200,2022-07-12
1,147980,962364,7,37000.000,2022-07-06,2022-07-13,37962.010,2022-07-13,2022-07-16,2022-10-14,2022-07-13,37962.000,2022-07-13


In [88]:
import datetime

In [89]:
df2['days_after_rollover'] = (df2['max_transaction_date'] - df2['end_rollover_dates']).days
df2.head(5)

AttributeError: 'Series' object has no attribute 'days'

In [83]:
def calc_trans_by_due_date(df):
    
    expected_matured_on_date = df['expected_matured_on_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= expected_matured_on_date:
        return 'yes'
    elif transaction_date > expected_matured_on_date:
        return 'no'
    
    
def calc_trans_by_end_rollover_date(df):
    
    end_rollover_dates = df['end_rollover_dates']
    transaction_date = df['transaction_date']
    
    if transaction_date <= end_rollover_dates:
        return 'yes'
    elif transaction_date > end_rollover_dates:
        return 'no'
    
    
def calc_trans_by_dpd_90_date(df):
    
    dpd_90_date = df['dpd_90_date']
    transaction_date = df['transaction_date']
    
    if transaction_date <= dpd_90_date:
        return 'yes'
    elif transaction_date > dpd_90_date:
        return 'no'

In [84]:
df2['transaction_by_due_date'] = df2.apply(lambda x: calc_trans_by_due_date(x), axis = 1)
df2['transaction_by_end_rollover_date'] = df2.apply(lambda x: calc_trans_by_end_rollover_date(x), axis = 1)
df2['transaction_by_dpd_90_date'] = df2.apply(lambda x: calc_trans_by_dpd_90_date(x), axis = 1)


df2.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,expected_matured_on_date,total_expected_repayment,due_date_fixed,end_rollover_dates,dpd_90_date,transaction_date,amount,transaction_by_due_date,transaction_by_end_rollover_date,transaction_by_dpd_90_date
0,81944,7681975,7,5000.000,2022-05-01,2022-05-08,5130.010,2022-05-08,2022-05-11,2022-08-09,2022-05-06,5000.000,yes,yes,yes
1,81944,7681975,7,5000.000,2022-05-01,2022-05-08,5130.010,2022-05-08,2022-05-11,2022-08-09,2022-05-06,130.000,yes,yes,yes


In [85]:
trans_by_due_date = df2[df2['transaction_by_due_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_trans_by_due_date = trans_by_due_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_trans_by_due_date = amount_trans_by_due_date.rename(columns={'amount':'amount_trans_by_due_date'})

amount_trans_by_due_date.head(2)

,loan_mifos_id,amount_trans_by_due_date
0,3697,155.100
1,3698,138.700


In [86]:
transaction_by_end_rollover_date = df2[df2['transaction_by_end_rollover_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_end_rollover_date = transaction_by_end_rollover_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_end_rollover_date = amount_transaction_by_end_rollover_date.rename(columns={'amount':'amount_transaction_by_end_rollover_date'})

amount_transaction_by_end_rollover_date.head(2)

,loan_mifos_id,amount_transaction_by_end_rollover_date
0,3697,155.100
1,3698,138.700


In [87]:
transaction_by_dpd_90_date = df2[df2['transaction_by_dpd_90_date'] == 'yes']

# grouping loan_mifos_id to find amount transacted by due date
amount_transaction_by_dpd_90_date = transaction_by_dpd_90_date.groupby(['loan_mifos_id'], as_index=False)['amount'].sum()
amount_transaction_by_dpd_90_date = amount_transaction_by_dpd_90_date.rename(columns={'amount':'amount_transaction_by_dpd_90_date'})

amount_transaction_by_dpd_90_date.head(2)

,loan_mifos_id,amount_transaction_by_dpd_90_date
0,3697,155.100
1,3698,138.700


In [88]:
final_df = pd.merge(amount_trans_by_due_date, amount_transaction_by_end_rollover_date, on = 'loan_mifos_id', how = 'outer')
final_df = pd.merge(final_df, amount_transaction_by_dpd_90_date, on = 'loan_mifos_id', how = 'outer')
final_df = final_df.fillna(0)

final_df.head(2)

,loan_mifos_id,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date
0,3697,155.100,155.100,155.100
1,3698,138.700,138.700,138.700


In [89]:
df_loans = df_loans[['loan_mifos_id', 'store_number', 'term_frequency', 'principal_disbursed', 'disbursed_on_date', 'total_expected_repayment']]
final_df = pd.merge(df_loans, final_df, on = 'loan_mifos_id', how = 'left')
final_df = final_df.fillna(0)

final_df.head(2)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date
0,81944,7681975,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000
1,81951,7102289,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000


In [90]:
final_df.head(15)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date
0,81944,7681975,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000
1,81951,7102289,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000
2,82073,7281864,7,12000.000,2022-05-01,12312.010,12312.000,12312.000,12312.000
3,82139,7284957,7,8700.000,2022-05-01,8926.210,8926.200,8926.200,8926.200
4,84385,539446,7,3700.000,2022-05-04,3796.210,3796.200,3796.200,3796.200
5,85451,292232,7,4000.000,2022-05-05,4104.010,4104.000,4104.000,4104.000
6,85444,7214025,7,2000.000,2022-04-30,2052.010,2052.000,2052.000,2052.000
7,75077,476789,7,1900.000,2022-04-22,2601.380,0.000,0.000,34.200
8,57694,7629469,1,200.000,2022-04-03,257.900,0.000,0.000,3.600
9,190875,919477,7,200000.000,2022-08-11,205200.010,0.000,0.000,0.000


In [91]:
final_df['repayments_ratio_by_due_date'] = final_df['amount_trans_by_due_date'] / final_df['total_expected_repayment']
final_df['repayments_ratio_by_end_rollover_date'] = final_df['amount_transaction_by_end_rollover_date'] / final_df['total_expected_repayment']
final_df['repayments_ratio_by_dpd_90_date'] = final_df['amount_transaction_by_dpd_90_date'] / final_df['total_expected_repayment']

final_df.head(10)

,loan_mifos_id,store_number,term_frequency,principal_disbursed,disbursed_on_date,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date
0,81944,7681975,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000,1.000,1.000,1.000
1,81951,7102289,7,5000.000,2022-05-01,5130.010,5130.000,5130.000,5130.000,1.000,1.000,1.000
2,82073,7281864,7,12000.000,2022-05-01,12312.010,12312.000,12312.000,12312.000,1.000,1.000,1.000
3,82139,7284957,7,8700.000,2022-05-01,8926.210,8926.200,8926.200,8926.200,1.000,1.000,1.000
4,84385,539446,7,3700.000,2022-05-04,3796.210,3796.200,3796.200,3796.200,1.000,1.000,1.000
5,85451,292232,7,4000.000,2022-05-05,4104.010,4104.000,4104.000,4104.000,1.000,1.000,1.000
6,85444,7214025,7,2000.000,2022-04-30,2052.010,2052.000,2052.000,2052.000,1.000,1.000,1.000
7,75077,476789,7,1900.000,2022-04-22,2601.380,0.000,0.000,34.200,0.000,0.000,0.013
8,57694,7629469,1,200.000,2022-04-03,257.900,0.000,0.000,3.600,0.000,0.000,0.014
9,190875,919477,7,200000.000,2022-08-11,205200.010,0.000,0.000,0.000,0.000,0.000,0.000


In [92]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175784 entries, 0 to 175783
Data columns (total 12 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   loan_mifos_id                            175784 non-null  int64         
 1   store_number                             175784 non-null  object        
 2   term_frequency                           175784 non-null  int64         
 3   principal_disbursed                      175784 non-null  float64       
 4   disbursed_on_date                        175784 non-null  datetime64[ns]
 5   total_expected_repayment                 175784 non-null  float64       
 6   amount_trans_by_due_date                 175784 non-null  float64       
 7   amount_transaction_by_end_rollover_date  175784 non-null  float64       
 8   amount_transaction_by_dpd_90_date        175784 non-null  float64       
 9   repayments_ratio_by_due_da

#### First March Refresh

##### 21 Day Loans

In [93]:
refresh_one_21_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-04') & (final_df['disbursed_on_date'] <= '2022-03-10') & (final_df['term_frequency'] == 21)]

refresh_one_21_day_loans = refresh_one_21_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_one_21_day_limits_and_loans = pd.merge(refresh_one_21_day_loans, refresh_one_21_day_limits, on = "store_number", how = 'left')

refresh_one_21_day_limits_and_loans['new_loan_amount_raw'] = refresh_one_21_day_limits_and_loans['principal_disbursed'] + refresh_one_21_day_limits_and_loans['limits_difference']

refresh_one_21_day_limits_and_loans['new_loan_amount'] = refresh_one_21_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_one_21_day_limits_and_loans['loan_difference'] = refresh_one_21_day_limits_and_loans['new_loan_amount'] - refresh_one_21_day_limits_and_loans['principal_disbursed']

refresh_one_21_day_limits_and_loans['facility_fee'] = refresh_one_21_day_limits_and_loans['new_loan_amount'] * 7.60/100
refresh_one_21_day_limits_and_loans['excise_duty'] = refresh_one_21_day_limits_and_loans['new_loan_amount'] * 1.52/100
refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_one_21_day_limits_and_loans['new_loan_amount'] + refresh_one_21_day_limits_and_loans['facility_fee'] + refresh_one_21_day_limits_and_loans['excise_duty']
#refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 106/100

refresh_one_21_day_limits_and_loans.head(5)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800


In [94]:
refresh_one_21_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_one_21_day_limits_and_loans['repayments_ratio_by_due_date']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772


In [95]:
refresh_one_21_day_limits_and_loans['rollover_amount'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_one_21_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_one_21_day_limits_and_loans['updated_rollover_amount'] = refresh_one_21_day_limits_and_loans['rollover_amount'] * 106/100

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030


In [96]:
refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_21_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_one_21_day_limits_and_loans['updated_rollover_amount']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802


In [97]:
refresh_one_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_one_21_day_limits_and_loans['repayments_ratio_by_end_rollover_date']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774


In [98]:
refresh_one_21_day_limits_and_loans['default_amount'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_one_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383,-168.364
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960,1392.258
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000,61881.952
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190,0.010
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774,0.028


In [99]:
def daily_default_charge(df):
    
    P = df['default_amount']
    r = 0.72/100
    n = 1
    t = 90

    if P >= 0:
        return P*(pow((1+r/n), n*t))

In [100]:
refresh_one_21_day_limits_and_loans['updated_default_amount'] = refresh_one_21_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)

refresh_one_21_day_limits_and_loans

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383,-168.364,NaN
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960,1392.258,2655.427
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000,61881.952,118026.232
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190,0.010,0.019
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774,0.028,0.053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,334734,177200.000,21,190667.210,190667.200,190667.200,190667.200,1.000,1.000,1.000,198700.000,43400.000,-155300.000,Decrease in Limits,21900.000,21900.000,-155300.000,1664.400,332.880,23897.280,23897.279,0.001,0.001,23897.280,23897.279,0.001,0.002
670,908488,10000.000,21,10760.010,10760.000,10760.000,10760.000,1.000,1.000,1.000,0.000,9600.000,9600.000,Increase in Limits,19600.000,19600.000,9600.000,1489.600,297.920,21387.520,21387.500,0.020,0.021,21387.521,21387.501,0.020,0.038
671,7064458,11000.000,21,11836.010,11836.000,11836.000,11836.000,1.000,1.000,1.000,0.000,48100.000,48100.000,Increase in Limits,59100.000,59100.000,48100.000,4491.600,898.320,64489.920,64489.866,0.054,0.058,64489.923,64489.869,0.054,0.104
672,7495177,90000.000,21,96840.010,96840.000,96840.000,96840.000,1.000,1.000,1.000,0.000,200000.000,200000.000,Increase in Limits,290000.000,290000.000,200000.000,22040.000,4408.000,316448.000,316447.967,0.033,0.035,316448.002,316447.969,0.033,0.062


In [101]:
refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383,-168.364,NaN
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960,1392.258,2655.427
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000,61881.952,118026.232
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190,0.010,0.019
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774,0.028,0.053


In [102]:
refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_one_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_one_21_day_limits_and_loans['updated_default_amount']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383,-168.364,NaN,NaN
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960,1392.258,2655.427,23170.387
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000,61881.952,118026.232,118026.232
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190,0.010,0.019,47467.210
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774,0.028,0.053,42556.827


In [103]:
refresh_one_21_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_one_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_one_21_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_one_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,614491,22000.000,21,24022.220,7490.000,25505.330,25505.330,0.312,1.062,1.062,19600.000,0.000,-19600.000,Decrease in Limits,2400.000,2400.000,-19600.000,182.400,36.480,2618.880,816.553,1802.327,1910.467,2727.020,2895.383,-168.364,NaN,NaN,NaN
1,615681,20000.000,21,22313.050,20895.000,20895.000,20943.270,0.936,0.936,0.939,0.000,0.000,0.000,No Change in Limits,20000.000,20000.000,0.000,1520.000,304.000,21824.000,20437.030,1386.970,1470.188,21907.218,20514.960,1392.258,2655.427,23170.387,21747.976
2,7168295,53500.000,21,78628.490,0.000,0.000,1926.000,0.000,0.000,0.024,0.000,0.000,0.000,No Change in Limits,53500.000,53500.000,0.000,4066.000,813.200,58379.200,0.000,58379.200,61881.952,61881.952,0.000,61881.952,118026.232,118026.232,2891.045
3,818115,43500.000,21,46806.010,46806.000,46806.000,46806.000,1.000,1.000,1.000,0.000,0.000,0.000,No Change in Limits,43500.000,43500.000,0.000,3306.000,661.200,47467.200,47467.190,0.010,0.011,47467.201,47467.190,0.010,0.019,47467.210,47467.200
4,7112836,14100.000,21,15171.610,15171.600,15171.600,15171.600,1.000,1.000,1.000,0.000,24900.000,24900.000,Increase in Limits,39000.000,39000.000,24900.000,2964.000,592.800,42556.800,42556.772,0.028,0.030,42556.802,42556.774,0.028,0.053,42556.827,42556.799


In [104]:
refresh_one_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  45455574.945
new_amount_transaction_by_end_rollover_date   47434803.780
new_amount_transaction_by_dpd_90_date         47872318.871
dtype: float64

In [105]:
refresh_one_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_one_21_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  1.002
new_amount_transaction_by_end_rollover_date   1.046
new_amount_transaction_by_dpd_90_date         1.056
dtype: float64

In [106]:
refresh_one_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  45455574.945
new_amount_transaction_by_end_rollover_date   47434803.780
new_amount_transaction_by_dpd_90_date         47872318.871
dtype: float64

In [107]:
refresh_one_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_one_21_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  1.002
new_amount_transaction_by_end_rollover_date   1.046
new_amount_transaction_by_dpd_90_date         1.056
dtype: float64

In [108]:
refresh_one_21_day_limits_and_loans['new_loan_amount'].sum()

45344450.0

##### 7 Day Loans

In [109]:
refresh_one_7_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-04') & (final_df['disbursed_on_date'] <= '2022-03-10') & (final_df['term_frequency'] == 7)]

refresh_one_7_day_loans = refresh_one_7_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_one_7_day_limits_and_loans = pd.merge(refresh_one_7_day_loans, refresh_one_7_day_limits, on = "store_number", how = 'left')

refresh_one_7_day_limits_and_loans['new_loan_amount_raw'] = refresh_one_7_day_limits_and_loans['principal_disbursed'] + refresh_one_7_day_limits_and_loans['limits_difference']

refresh_one_7_day_limits_and_loans['new_loan_amount'] = refresh_one_7_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_one_7_day_limits_and_loans['loan_difference'] = refresh_one_7_day_limits_and_loans['new_loan_amount'] - refresh_one_7_day_limits_and_loans['principal_disbursed']

refresh_one_7_day_limits_and_loans['facility_fee'] = refresh_one_7_day_limits_and_loans['new_loan_amount'] * 2.60/100
refresh_one_7_day_limits_and_loans['excise_duty'] = refresh_one_7_day_limits_and_loans['new_loan_amount'] * 0.52/100
refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_one_7_day_limits_and_loans['new_loan_amount'] + refresh_one_7_day_limits_and_loans['facility_fee'] + refresh_one_7_day_limits_and_loans['excise_duty']
#refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 103/100

refresh_one_7_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7511585,4100.000,7,4402.700,1500.000,1500.000,4708.200,0.341,0.341,1.069,11300.000,0.000,-11300.000,Decrease in Limits,-7200.000,0.000,-4100.000,0.000,0.000,0.000
1,735947,6600.000,7,6956.600,2087.000,2087.000,6974.070,0.300,0.300,1.003,8200.000,7000.000,-1200.000,Decrease in Limits,5400.000,5400.000,-1200.000,140.400,28.080,5568.480


In [110]:
refresh_one_7_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_one_7_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_one_7_day_limits_and_loans['rollover_amount'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_one_7_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_one_7_day_limits_and_loans['updated_rollover_amount'] = refresh_one_7_day_limits_and_loans['rollover_amount'] * 103/100
refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_7_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_one_7_day_limits_and_loans['updated_rollover_amount']

refresh_one_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_one_7_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_one_7_day_limits_and_loans['default_amount'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_one_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_one_7_day_limits_and_loans['updated_default_amount'] = refresh_one_7_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_one_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_one_7_day_limits_and_loans['updated_default_amount']

refresh_one_7_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_one_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_one_7_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_one_7_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7511585,4100.000,7,4402.700,1500.000,1500.000,4708.200,0.341,0.341,1.069,11300.000,0.000,-11300.000,Decrease in Limits,-7200.000,0.000,-4100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,735947,6600.000,7,6956.600,2087.000,2087.000,6974.070,0.300,0.300,1.003,8200.000,7000.000,-1200.000,Decrease in Limits,5400.000,5400.000,-1200.000,140.400,28.080,5568.480,1670.560,3897.920,4014.858,5685.418,1705.642,3979.776,7590.549,9296.190,9319.536
2,7963735,3600.000,7,3813.130,3693.600,3693.600,3813.130,0.969,0.969,1.000,4400.000,0.000,-4400.000,Decrease in Limits,-800.000,0.000,-3600.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,7230988,4800.000,7,5131.210,0.000,0.000,5160.000,0.000,0.000,1.006,5900.000,1700.000,-4200.000,Decrease in Limits,600.000,600.000,-4200.000,15.600,3.120,618.720,0.000,618.720,637.282,637.282,0.000,637.282,1215.475,1215.475,1222.294
4,972636,22000.000,7,22952.410,0.000,23122.000,23122.000,0.000,1.007,1.007,NaN,NaN,NaN,NaN,NaN,0.000,-22000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [111]:
refresh_one_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  28675552.840
new_amount_transaction_by_end_rollover_date   31206921.149
new_amount_transaction_by_dpd_90_date         32161218.153
dtype: float64

In [112]:
refresh_one_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_one_7_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.920
new_amount_transaction_by_end_rollover_date   1.002
new_amount_transaction_by_dpd_90_date         1.032
dtype: float64

In [113]:
refresh_one_7_day_limits_and_loans['new_loan_amount'].sum()

31155887.0

##### 1 Day Loans

In [114]:
refresh_one_1_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-04') & (final_df['disbursed_on_date'] <= '2022-03-10') & (final_df['term_frequency'] == 1)]

refresh_one_1_day_loans = refresh_one_1_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_one_1_day_limits_and_loans = pd.merge(refresh_one_1_day_loans, refresh_one_1_day_limits, on = "store_number", how = 'left')

refresh_one_1_day_limits_and_loans['new_loan_amount_raw'] = refresh_one_1_day_limits_and_loans['principal_disbursed'] + refresh_one_1_day_limits_and_loans['limits_difference']

refresh_one_1_day_limits_and_loans['new_loan_amount'] = refresh_one_1_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_one_1_day_limits_and_loans['loan_difference'] = refresh_one_1_day_limits_and_loans['new_loan_amount'] - refresh_one_1_day_limits_and_loans['principal_disbursed']

refresh_one_1_day_limits_and_loans['facility_fee'] = refresh_one_1_day_limits_and_loans['new_loan_amount'] * 0.40/100
refresh_one_1_day_limits_and_loans['excise_duty'] = refresh_one_1_day_limits_and_loans['new_loan_amount'] * 0.08/100
refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_one_1_day_limits_and_loans['new_loan_amount'] + refresh_one_1_day_limits_and_loans['facility_fee'] + refresh_one_1_day_limits_and_loans['excise_duty']
#refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 100.48/100

refresh_one_1_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7628381,3100.000,1,3162.090,0.000,0.000,3162.090,0.000,0.000,1.000,8200.000,0.000,-8200.000,Decrease in Limits,-5100.000,0.000,-3100.000,0.000,0.000,0.000
1,7464929,800.000,1,809.610,0.000,809.200,809.200,0.000,0.999,0.999,1400.000,0.000,-1400.000,Decrease in Limits,-600.000,0.000,-800.000,0.000,0.000,0.000


In [115]:
refresh_one_1_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_one_1_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_one_1_day_limits_and_loans['rollover_amount'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_one_1_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_one_1_day_limits_and_loans['updated_rollover_amount'] = refresh_one_1_day_limits_and_loans['rollover_amount'] * 100.48/100
refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_one_1_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_one_1_day_limits_and_loans['updated_rollover_amount']

refresh_one_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_one_1_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_one_1_day_limits_and_loans['default_amount'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_one_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_one_1_day_limits_and_loans['updated_default_amount'] = refresh_one_1_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_one_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_one_1_day_limits_and_loans['updated_default_amount']

refresh_one_1_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_one_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_one_1_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_one_1_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7628381,3100.000,1,3162.090,0.000,0.000,3162.090,0.000,0.000,1.000,8200.000,0.000,-8200.000,Decrease in Limits,-5100.000,0.000,-3100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7464929,800.000,1,809.610,0.000,809.200,809.200,0.000,0.999,0.999,1400.000,0.000,-1400.000,Decrease in Limits,-600.000,0.000,-800.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,7824617,500.000,1,504.130,502.000,502.000,504.130,0.996,0.996,1.000,1100.000,1200.000,100.000,Increase in Limits,600.000,600.000,100.000,2.400,0.480,602.880,600.333,2.547,2.559,602.892,600.345,2.547,4.858,605.203,605.203
3,511449,2500.000,1,2551.490,2510.000,2510.000,2551.490,0.984,0.984,1.000,3000.000,1800.000,-1200.000,Decrease in Limits,1300.000,1300.000,-1200.000,5.200,1.040,1306.240,1284.999,21.241,21.343,1306.342,1285.099,21.243,40.515,1325.615,1325.615
4,7824617,500.000,1,504.120,502.000,502.000,504.120,0.996,0.996,1.000,1100.000,1200.000,100.000,Increase in Limits,600.000,600.000,100.000,2.400,0.480,602.880,600.345,2.535,2.547,602.892,600.357,2.535,4.836,605.192,605.192


In [116]:
refresh_one_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  1068188.770
new_amount_transaction_by_end_rollover_date   1213204.129
new_amount_transaction_by_dpd_90_date         1603622.388
dtype: float64

In [117]:
refresh_one_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_one_1_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.748
new_amount_transaction_by_end_rollover_date   0.850
new_amount_transaction_by_dpd_90_date         1.123
dtype: float64

In [118]:
refresh_one_1_day_limits_and_loans['new_loan_amount'].sum()

1427800.0

#### Second March Refresh

##### 21 Day Loans

In [119]:
refresh_two_21_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-11') & (final_df['disbursed_on_date'] <= '2022-03-24') & (final_df['term_frequency'] == 21)]

refresh_two_21_day_loans = refresh_two_21_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_two_21_day_limits_and_loans = pd.merge(refresh_two_21_day_loans, refresh_two_21_day_limits, on = "store_number", how = 'left')

refresh_two_21_day_limits_and_loans['new_loan_amount_raw'] = refresh_two_21_day_limits_and_loans['principal_disbursed'] + refresh_two_21_day_limits_and_loans['limits_difference']

refresh_two_21_day_limits_and_loans['new_loan_amount'] = refresh_two_21_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_two_21_day_limits_and_loans['loan_difference'] = refresh_two_21_day_limits_and_loans['new_loan_amount'] - refresh_two_21_day_limits_and_loans['principal_disbursed']

refresh_two_21_day_limits_and_loans['facility_fee'] = refresh_two_21_day_limits_and_loans['new_loan_amount'] * 7.60/100
refresh_two_21_day_limits_and_loans['excise_duty'] = refresh_two_21_day_limits_and_loans['new_loan_amount'] * 1.52/100
refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_two_21_day_limits_and_loans['new_loan_amount'] + refresh_two_21_day_limits_and_loans['facility_fee'] + refresh_two_21_day_limits_and_loans['excise_duty']
#refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 106/100

refresh_two_21_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,871756,60000.000,21,67740.010,0.000,0.000,71959.040,0.000,0.000,1.062,0.000,32800.000,32800.000,Increase in Limits,92800.000,92800.000,32800.000,7052.800,1410.560,101263.360
1,583345,2800.000,21,3155.610,0.000,3246.130,3246.130,0.000,1.029,1.029,0.000,0.000,0.000,No Change in Limits,2800.000,2800.000,0.000,212.800,42.560,3055.360


In [120]:
refresh_two_21_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_two_21_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_two_21_day_limits_and_loans['rollover_amount'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_two_21_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_two_21_day_limits_and_loans['updated_rollover_amount'] = refresh_two_21_day_limits_and_loans['rollover_amount'] * 106/100
refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_21_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_two_21_day_limits_and_loans['updated_rollover_amount']

refresh_two_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_two_21_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_two_21_day_limits_and_loans['default_amount'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_two_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_two_21_day_limits_and_loans['updated_default_amount'] = refresh_two_21_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_two_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_two_21_day_limits_and_loans['updated_default_amount']

refresh_two_21_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_two_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_two_21_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_two_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,871756,60000.000,21,67740.010,0.000,0.000,71959.040,0.000,0.000,1.062,0.000,32800.000,32800.000,Increase in Limits,92800.000,92800.000,32800.000,7052.800,1410.560,101263.360,0.000,101263.360,107339.162,107339.162,0.000,107339.162,204725.876,204725.876,217476.754
1,583345,2800.000,21,3155.610,0.000,3246.130,3246.130,0.000,1.029,1.029,0.000,0.000,0.000,No Change in Limits,2800.000,2800.000,0.000,212.800,42.560,3055.360,0.000,3055.360,3238.682,3238.682,3331.585,-92.903,NaN,NaN,NaN
2,982659,100000.000,21,110322.810,0.000,0.000,112100.000,0.000,0.000,1.016,130300.000,106700.000,-23600.000,Decrease in Limits,76400.000,76400.000,-23600.000,5806.400,1161.280,83367.680,0.000,83367.680,88369.741,88369.741,0.000,88369.741,168545.872,168545.872,171260.977
3,667381,26400.000,21,45108.610,0.000,3000.000,3950.400,0.000,0.067,0.088,26400.000,0.000,-26400.000,Decrease in Limits,0.000,0.000,-26400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,580441,4000.000,21,7074.410,0.000,0.000,144.000,0.000,0.000,0.020,0.000,0.000,0.000,No Change in Limits,4000.000,4000.000,0.000,304.000,60.800,4364.800,0.000,4364.800,4626.688,4626.688,0.000,4626.688,8824.391,8824.391,179.621


In [121]:
refresh_two_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  59149259.457
new_amount_transaction_by_end_rollover_date   62836814.171
new_amount_transaction_by_dpd_90_date         62649983.070
dtype: float64

In [122]:
refresh_two_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_two_21_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  1.001
new_amount_transaction_by_end_rollover_date   1.064
new_amount_transaction_by_dpd_90_date         1.061
dtype: float64

In [123]:
refresh_two_21_day_limits_and_loans['new_loan_amount'].sum()

59073340.0

##### 7 Day Loans

In [124]:
refresh_two_7_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-11') & (final_df['disbursed_on_date'] <= '2022-03-24') & (final_df['term_frequency'] == 7)]

refresh_two_7_day_loans = refresh_two_7_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_two_7_day_limits_and_loans = pd.merge(refresh_two_7_day_loans, refresh_two_7_day_limits, on = "store_number", how = 'left')

refresh_two_7_day_limits_and_loans['new_loan_amount_raw'] = refresh_two_7_day_limits_and_loans['principal_disbursed'] + refresh_two_7_day_limits_and_loans['limits_difference']

refresh_two_7_day_limits_and_loans['new_loan_amount'] = refresh_two_7_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_two_7_day_limits_and_loans['loan_difference'] = refresh_two_7_day_limits_and_loans['new_loan_amount'] - refresh_two_7_day_limits_and_loans['principal_disbursed']

refresh_two_7_day_limits_and_loans['facility_fee'] = refresh_two_7_day_limits_and_loans['new_loan_amount'] * 2.60/100
refresh_two_7_day_limits_and_loans['excise_duty'] = refresh_two_7_day_limits_and_loans['new_loan_amount'] * 0.52/100
refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_two_7_day_limits_and_loans['new_loan_amount'] + refresh_two_7_day_limits_and_loans['facility_fee'] + refresh_two_7_day_limits_and_loans['excise_duty']
#refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 103/100

refresh_two_7_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7553643,2200.000,7,2963.410,0.000,0.000,26.420,0.000,0.000,0.009,2800.000,0.000,-2800.000,Decrease in Limits,-600.000,0.000,-2200.000,0.000,0.000,0.000
1,7626883,5000.000,7,6595.010,0.000,0.000,0.000,0.000,0.000,0.000,5000.000,0.000,-5000.000,Decrease in Limits,0.000,0.000,-5000.000,0.000,0.000,0.000


In [125]:
refresh_two_7_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_two_7_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_two_7_day_limits_and_loans['rollover_amount'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_two_7_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_two_7_day_limits_and_loans['updated_rollover_amount'] = refresh_two_7_day_limits_and_loans['rollover_amount'] * 103/100
refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_7_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_two_7_day_limits_and_loans['updated_rollover_amount']

refresh_two_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_two_7_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_two_7_day_limits_and_loans['default_amount'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_two_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_two_7_day_limits_and_loans['updated_default_amount'] = refresh_two_7_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_two_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_two_7_day_limits_and_loans['updated_default_amount']

refresh_two_7_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_two_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_two_7_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_two_7_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7553643,2200.000,7,2963.410,0.000,0.000,26.420,0.000,0.000,0.009,2800.000,0.000,-2800.000,Decrease in Limits,-600.000,0.000,-2200.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7626883,5000.000,7,6595.010,0.000,0.000,0.000,0.000,0.000,0.000,5000.000,0.000,-5000.000,Decrease in Limits,0.000,0.000,-5000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,7017311,1500.000,7,1978.510,0.000,0.000,0.000,0.000,0.000,0.000,1900.000,0.000,-1900.000,Decrease in Limits,-400.000,0.000,-1500.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,7584077,15300.000,7,20364.310,0.000,0.000,0.000,0.000,0.000,0.000,5700.000,0.000,-5700.000,Decrease in Limits,9600.000,9600.000,-5700.000,249.600,49.920,9899.520,0.000,9899.520,10196.506,10196.506,0.000,10196.506,19447.595,19447.595,0.000
4,630904,14000.000,7,15109.820,0.000,0.000,16947.190,0.000,0.000,1.122,17300.000,4400.000,-12900.000,Decrease in Limits,1100.000,1100.000,-12900.000,28.600,5.720,1134.320,0.000,1134.320,1168.350,1168.350,0.000,1168.350,2228.370,2228.370,2499.342


In [126]:
refresh_two_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  81994938.417
new_amount_transaction_by_end_rollover_date   89350247.708
new_amount_transaction_by_dpd_90_date         91817901.997
dtype: float64

In [127]:
refresh_two_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_two_7_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.918
new_amount_transaction_by_end_rollover_date   1.000
new_amount_transaction_by_dpd_90_date         1.028
dtype: float64

In [128]:
refresh_two_7_day_limits_and_loans['new_loan_amount'].sum()

89348130.0

##### 1 Day Loans

In [129]:
refresh_two_1_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-11') & (final_df['disbursed_on_date'] <= '2022-03-24') & (final_df['term_frequency'] == 1)]

refresh_two_1_day_loans = refresh_two_1_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_two_1_day_limits_and_loans = pd.merge(refresh_two_1_day_loans, refresh_two_1_day_limits, on = "store_number", how = 'left')

refresh_two_1_day_limits_and_loans['new_loan_amount_raw'] = refresh_two_1_day_limits_and_loans['principal_disbursed'] + refresh_two_1_day_limits_and_loans['limits_difference']

refresh_two_1_day_limits_and_loans['new_loan_amount'] = refresh_two_1_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_two_1_day_limits_and_loans['loan_difference'] = refresh_two_1_day_limits_and_loans['new_loan_amount'] - refresh_two_1_day_limits_and_loans['principal_disbursed']

refresh_two_1_day_limits_and_loans['facility_fee'] = refresh_two_1_day_limits_and_loans['new_loan_amount'] * 0.40/100
refresh_two_1_day_limits_and_loans['excise_duty'] = refresh_two_1_day_limits_and_loans['new_loan_amount'] * 0.08/100
refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_two_1_day_limits_and_loans['new_loan_amount'] + refresh_two_1_day_limits_and_loans['facility_fee'] + refresh_two_1_day_limits_and_loans['excise_duty']
#refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 100.48/100

refresh_two_1_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7464929,800.000,1,815.600,0.000,0.000,815.600,0.000,0.000,1.000,1400.000,0.000,-1400.000,Decrease in Limits,-600.000,0.000,-800.000,0.000,0.000,0.000
1,7285467,10000.000,1,10242.550,10040.000,10040.000,10242.550,0.980,0.980,1.000,54300.000,68100.000,13800.000,Increase in Limits,23800.000,23800.000,13800.000,95.200,19.040,23914.240


In [130]:
refresh_two_1_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_two_1_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_two_1_day_limits_and_loans['rollover_amount'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_two_1_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_two_1_day_limits_and_loans['updated_rollover_amount'] = refresh_two_1_day_limits_and_loans['rollover_amount'] * 100.48/100
refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_two_1_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_two_1_day_limits_and_loans['updated_rollover_amount']

refresh_two_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_two_1_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_two_1_day_limits_and_loans['default_amount'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_two_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_two_1_day_limits_and_loans['updated_default_amount'] = refresh_two_1_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_two_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_two_1_day_limits_and_loans['updated_default_amount']

refresh_two_1_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_two_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_two_1_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_two_1_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7464929,800.000,1,815.600,0.000,0.000,815.600,0.000,0.000,1.000,1400.000,0.000,-1400.000,Decrease in Limits,-600.000,0.000,-800.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7285467,10000.000,1,10242.550,10040.000,10040.000,10242.550,0.980,0.980,1.000,54300.000,68100.000,13800.000,Increase in Limits,23800.000,23800.000,13800.000,95.200,19.040,23914.240,23441.328,472.912,475.182,23916.510,23443.553,472.957,902.062,24345.615,24345.615
2,7156529,400.000,1,401.600,401.600,401.600,401.600,1.000,1.000,1.000,4800.000,0.000,-4800.000,Decrease in Limits,-4400.000,0.000,-400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,7276775,1700.000,1,1713.720,1706.800,1706.800,1713.720,0.996,0.996,1.000,1900.000,4500.000,2600.000,Increase in Limits,4300.000,4300.000,2600.000,17.200,3.440,4320.640,4303.193,17.447,17.530,4320.724,4303.277,17.447,33.276,4336.553,4336.553
4,7285467,1000.000,1,1004.000,1004.000,1004.000,1004.000,1.000,1.000,1.000,54300.000,68100.000,13800.000,Increase in Limits,14800.000,14800.000,13800.000,59.200,11.840,14871.040,14871.040,0.000,0.000,14871.040,14871.040,0.000,0.000,14871.040,14871.040


In [131]:
refresh_two_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  5869823.951
new_amount_transaction_by_end_rollover_date   6628200.522
new_amount_transaction_by_dpd_90_date         6958845.362
dtype: float64

In [132]:
refresh_two_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_two_1_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.848
new_amount_transaction_by_end_rollover_date   0.957
new_amount_transaction_by_dpd_90_date         1.005
dtype: float64

In [133]:
refresh_two_1_day_limits_and_loans['new_loan_amount'].sum()

6924400.0

#### Third March Refresh

##### 21 Day Loans

In [134]:
refresh_three_21_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-25') & (final_df['disbursed_on_date'] <= '2022-03-30') & (final_df['term_frequency'] == 21)]

refresh_three_21_day_loans = refresh_three_21_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_three_21_day_limits_and_loans = pd.merge(refresh_three_21_day_loans, refresh_three_21_day_limits, on = "store_number", how = 'left')

refresh_three_21_day_limits_and_loans['new_loan_amount_raw'] = refresh_three_21_day_limits_and_loans['principal_disbursed'] + refresh_three_21_day_limits_and_loans['limits_difference']

refresh_three_21_day_limits_and_loans['new_loan_amount'] = refresh_three_21_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_three_21_day_limits_and_loans['loan_difference'] = refresh_three_21_day_limits_and_loans['new_loan_amount'] - refresh_three_21_day_limits_and_loans['principal_disbursed']

refresh_three_21_day_limits_and_loans['facility_fee'] = refresh_three_21_day_limits_and_loans['new_loan_amount'] * 7.60/100
refresh_three_21_day_limits_and_loans['excise_duty'] = refresh_three_21_day_limits_and_loans['new_loan_amount'] * 1.52/100
refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_three_21_day_limits_and_loans['new_loan_amount'] + refresh_three_21_day_limits_and_loans['facility_fee'] + refresh_three_21_day_limits_and_loans['excise_duty']
#refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 106/100

refresh_three_21_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,263761,41000.000,21,44116.010,44116.000,44116.000,44116.000,1.000,1.000,1.000,46400.000,0.000,-46400.000,Decrease in Limits,-5400.000,0.000,-41000.000,0.000,0.000,0.000
1,947543,38000.000,21,40888.010,40888.000,40888.000,40888.000,1.000,1.000,1.000,38000.000,0.000,-38000.000,Decrease in Limits,0.000,0.000,-38000.000,0.000,0.000,0.000


In [135]:
refresh_three_21_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_three_21_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_three_21_day_limits_and_loans['rollover_amount'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_three_21_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_three_21_day_limits_and_loans['updated_rollover_amount'] = refresh_three_21_day_limits_and_loans['rollover_amount'] * 106/100
refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_21_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_three_21_day_limits_and_loans['updated_rollover_amount']

refresh_three_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_three_21_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_three_21_day_limits_and_loans['default_amount'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_three_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_three_21_day_limits_and_loans['updated_default_amount'] = refresh_three_21_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_three_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_three_21_day_limits_and_loans['updated_default_amount']

refresh_three_21_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_three_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_three_21_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_three_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,263761,41000.000,21,44116.010,44116.000,44116.000,44116.000,1.000,1.000,1.000,46400.000,0.000,-46400.000,Decrease in Limits,-5400.000,0.000,-41000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,947543,38000.000,21,40888.010,40888.000,40888.000,40888.000,1.000,1.000,1.000,38000.000,0.000,-38000.000,Decrease in Limits,0.000,0.000,-38000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,797049,24600.000,21,26877.430,26469.600,26469.600,26877.430,0.985,0.985,1.000,27200.000,0.000,-27200.000,Decrease in Limits,-2600.000,0.000,-24600.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,504471,180000.000,21,196598.240,193680.000,193680.000,196598.240,0.985,0.985,1.000,194600.000,140500.000,-54100.000,Decrease in Limits,125900.000,125900.000,-54100.000,9568.400,1913.680,137382.080,135342.825,2039.255,2161.610,137504.435,135463.364,2041.071,3892.894,139356.259,139356.259
4,7039976,46400.000,21,50971.950,49926.400,49926.400,50971.950,0.979,0.979,1.000,46400.000,0.000,-46400.000,Decrease in Limits,0.000,0.000,-46400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [136]:
refresh_three_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  27203974.733
new_amount_transaction_by_end_rollover_date   29527984.959
new_amount_transaction_by_dpd_90_date         29505975.924
dtype: float64

In [137]:
refresh_three_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_three_21_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.981
new_amount_transaction_by_end_rollover_date   1.064
new_amount_transaction_by_dpd_90_date         1.063
dtype: float64

In [138]:
refresh_three_21_day_limits_and_loans['new_loan_amount'].sum()

27744790.0

##### 7 Day Loans

In [139]:
refresh_three_7_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-25') & (final_df['disbursed_on_date'] <= '2022-03-30') & (final_df['term_frequency'] == 7)]

refresh_three_7_day_loans = refresh_three_7_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_three_7_day_limits_and_loans = pd.merge(refresh_three_7_day_loans, refresh_three_7_day_limits, on = "store_number", how = 'left')

refresh_three_7_day_limits_and_loans['new_loan_amount_raw'] = refresh_three_7_day_limits_and_loans['principal_disbursed'] + refresh_three_7_day_limits_and_loans['limits_difference']

refresh_three_7_day_limits_and_loans['new_loan_amount'] = refresh_three_7_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_three_7_day_limits_and_loans['loan_difference'] = refresh_three_7_day_limits_and_loans['new_loan_amount'] - refresh_three_7_day_limits_and_loans['principal_disbursed']

refresh_three_7_day_limits_and_loans['facility_fee'] = refresh_three_7_day_limits_and_loans['new_loan_amount'] * 2.60/100
refresh_three_7_day_limits_and_loans['excise_duty'] = refresh_three_7_day_limits_and_loans['new_loan_amount'] * 0.52/100
refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_three_7_day_limits_and_loans['new_loan_amount'] + refresh_three_7_day_limits_and_loans['facility_fee'] + refresh_three_7_day_limits_and_loans['excise_duty']
#refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 103/100

refresh_three_7_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7832153,5400.000,7,5664.470,150.000,5675.400,5675.400,0.026,1.002,1.002,6700.000,0.000,-6700.000,Decrease in Limits,-1300.000,0.000,-5400.000,0.000,0.000,0.000
1,7517459,2200.000,7,2321.020,0.000,0.000,2435.400,0.000,0.000,1.049,3200.000,0.000,-3200.000,Decrease in Limits,-1000.000,0.000,-2200.000,0.000,0.000,0.000


In [140]:
refresh_three_7_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_three_7_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_three_7_day_limits_and_loans['rollover_amount'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_three_7_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_three_7_day_limits_and_loans['updated_rollover_amount'] = refresh_three_7_day_limits_and_loans['rollover_amount'] * 103/100
refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_7_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_three_7_day_limits_and_loans['updated_rollover_amount']

refresh_three_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_three_7_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_three_7_day_limits_and_loans['default_amount'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_three_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_three_7_day_limits_and_loans['updated_default_amount'] = refresh_three_7_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_three_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_three_7_day_limits_and_loans['updated_default_amount']

refresh_three_7_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_three_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_three_7_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_three_7_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7832153,5400.000,7,5664.470,150.000,5675.400,5675.400,0.026,1.002,1.002,6700.000,0.000,-6700.000,Decrease in Limits,-1300.000,0.000,-5400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7517459,2200.000,7,2321.020,0.000,0.000,2435.400,0.000,0.000,1.049,3200.000,0.000,-3200.000,Decrease in Limits,-1000.000,0.000,-2200.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,7249705,1200.000,7,1258.810,0.000,0.000,1270.800,0.000,0.000,1.010,2800.000,10200.000,7400.000,Increase in Limits,8600.000,8600.000,7400.000,223.600,44.720,8868.320,0.000,8868.320,9134.370,9134.370,0.000,9134.370,17421.804,17421.804,17587.744
3,7036044,6800.000,7,8928.420,0.000,0.000,81.620,0.000,0.000,0.009,7200.000,0.000,-7200.000,Decrease in Limits,-400.000,0.000,-6800.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,7926125,6000.000,7,7878.740,0.000,0.000,680.000,0.000,0.000,0.086,6000.000,2200.000,-3800.000,Decrease in Limits,2200.000,2200.000,-3800.000,57.200,11.440,2268.640,0.000,2268.640,2336.699,2336.699,0.000,2336.699,4456.741,4456.741,384.653


In [141]:
refresh_three_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  36074634.262
new_amount_transaction_by_end_rollover_date   39951127.020
new_amount_transaction_by_dpd_90_date         39501896.236
dtype: float64

In [142]:
refresh_three_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_three_7_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.921
new_amount_transaction_by_end_rollover_date   1.020
new_amount_transaction_by_dpd_90_date         1.009
dtype: float64

In [143]:
refresh_three_7_day_limits_and_loans['new_loan_amount'].sum()

39155450.0

##### 1 Day Loans

In [144]:
refresh_three_1_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-25') & (final_df['disbursed_on_date'] <= '2022-03-30') & (final_df['term_frequency'] == 1)]

refresh_three_1_day_loans = refresh_three_1_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_three_1_day_limits_and_loans = pd.merge(refresh_three_1_day_loans, refresh_three_1_day_limits, on = "store_number", how = 'left')

refresh_three_1_day_limits_and_loans['new_loan_amount_raw'] = refresh_three_1_day_limits_and_loans['principal_disbursed'] + refresh_three_1_day_limits_and_loans['limits_difference']

refresh_three_1_day_limits_and_loans['new_loan_amount'] = refresh_three_1_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_three_1_day_limits_and_loans['loan_difference'] = refresh_three_1_day_limits_and_loans['new_loan_amount'] - refresh_three_1_day_limits_and_loans['principal_disbursed']

refresh_three_1_day_limits_and_loans['facility_fee'] = refresh_three_1_day_limits_and_loans['new_loan_amount'] * 0.40/100
refresh_three_1_day_limits_and_loans['excise_duty'] = refresh_three_1_day_limits_and_loans['new_loan_amount'] * 0.08/100
refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_three_1_day_limits_and_loans['new_loan_amount'] + refresh_three_1_day_limits_and_loans['facility_fee'] + refresh_three_1_day_limits_and_loans['excise_duty']
#refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 100.48/100

refresh_three_1_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7268306,900.000,1,1144.800,0.000,0.000,21.600,0.000,0.000,0.019,900.000,0.000,-900.000,Decrease in Limits,0.000,0.000,-900.000,0.000,0.000,0.000
1,526968,18000.000,1,18072.000,18072.000,18072.000,18072.000,1.000,1.000,1.000,93400.000,98600.000,5200.000,Increase in Limits,23200.000,23200.000,5200.000,92.800,18.560,23311.360


In [145]:
refresh_three_1_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_three_1_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_three_1_day_limits_and_loans['rollover_amount'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_three_1_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_three_1_day_limits_and_loans['updated_rollover_amount'] = refresh_three_1_day_limits_and_loans['rollover_amount'] * 100.48/100
refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_three_1_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_three_1_day_limits_and_loans['updated_rollover_amount']

refresh_three_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_three_1_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_three_1_day_limits_and_loans['default_amount'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_three_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_three_1_day_limits_and_loans['updated_default_amount'] = refresh_three_1_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_three_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_three_1_day_limits_and_loans['updated_default_amount']

refresh_three_1_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_three_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_three_1_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_three_1_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7268306,900.000,1,1144.800,0.000,0.000,21.600,0.000,0.000,0.019,900.000,0.000,-900.000,Decrease in Limits,0.000,0.000,-900.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,526968,18000.000,1,18072.000,18072.000,18072.000,18072.000,1.000,1.000,1.000,93400.000,98600.000,5200.000,Increase in Limits,23200.000,23200.000,5200.000,92.800,18.560,23311.360,23311.360,0.000,0.000,23311.360,23311.360,0.000,0.000,23311.360,23311.360
2,7187977,4800.000,1,4819.200,4819.200,4819.200,4819.200,1.000,1.000,1.000,4100.000,0.000,-4100.000,Decrease in Limits,700.000,700.000,-4100.000,2.800,0.560,703.360,703.360,0.000,0.000,703.360,703.360,0.000,0.000,703.360,703.360
3,7376916,500.000,1,502.000,502.000,502.000,502.000,1.000,1.000,1.000,500.000,0.000,-500.000,Decrease in Limits,0.000,0.000,-500.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,7376916,500.000,1,502.000,502.000,502.000,502.000,1.000,1.000,1.000,500.000,0.000,-500.000,Decrease in Limits,0.000,0.000,-500.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [146]:
refresh_three_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  2165088.272
new_amount_transaction_by_end_rollover_date   2197932.824
new_amount_transaction_by_dpd_90_date         2664404.933
dtype: float64

In [147]:
refresh_three_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_three_1_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.883
new_amount_transaction_by_end_rollover_date   0.896
new_amount_transaction_by_dpd_90_date         1.087
dtype: float64

In [148]:
refresh_three_1_day_limits_and_loans['new_loan_amount'].sum()

2452020.0

#### Fourth March Refresh

##### 21 Day Loans

In [149]:
refresh_four_21_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-31') & (final_df['disbursed_on_date'] <= '2022-04-07') & (final_df['term_frequency'] == 21)]

refresh_four_21_day_loans = refresh_four_21_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_four_21_day_limits_and_loans = pd.merge(refresh_four_21_day_loans, refresh_four_21_day_limits, on = "store_number", how = 'left')

refresh_four_21_day_limits_and_loans['new_loan_amount_raw'] = refresh_four_21_day_limits_and_loans['principal_disbursed'] + refresh_four_21_day_limits_and_loans['limits_difference']

refresh_four_21_day_limits_and_loans['new_loan_amount'] = refresh_four_21_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_four_21_day_limits_and_loans['loan_difference'] = refresh_four_21_day_limits_and_loans['new_loan_amount'] - refresh_four_21_day_limits_and_loans['principal_disbursed']

refresh_four_21_day_limits_and_loans['facility_fee'] = refresh_four_21_day_limits_and_loans['new_loan_amount'] * 7.60/100
refresh_four_21_day_limits_and_loans['excise_duty'] = refresh_four_21_day_limits_and_loans['new_loan_amount'] * 1.52/100
refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_four_21_day_limits_and_loans['new_loan_amount'] + refresh_four_21_day_limits_and_loans['facility_fee'] + refresh_four_21_day_limits_and_loans['excise_duty']
#refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 106/100

refresh_four_21_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,595541,25000.000,21,27213.240,26900.000,26900.000,27213.240,0.988,0.988,1.000,69500.000,38000.000,-31500.000,Decrease in Limits,-6500.000,0.000,-25000.000,0.000,0.000,0.000
1,7112836,26900.000,21,29367.580,28944.400,28944.400,29367.580,0.986,0.986,1.000,26900.000,24900.000,-2000.000,Decrease in Limits,24900.000,24900.000,-2000.000,1892.400,378.480,27170.880


In [150]:
refresh_four_21_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_four_21_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_four_21_day_limits_and_loans['rollover_amount'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_four_21_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_four_21_day_limits_and_loans['updated_rollover_amount'] = refresh_four_21_day_limits_and_loans['rollover_amount'] * 106/100
refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_21_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_four_21_day_limits_and_loans['updated_rollover_amount']

refresh_four_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_four_21_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_four_21_day_limits_and_loans['default_amount'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_four_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_four_21_day_limits_and_loans['updated_default_amount'] = refresh_four_21_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_four_21_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_four_21_day_limits_and_loans['updated_default_amount']

refresh_four_21_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_four_21_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_four_21_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_four_21_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_21_limit,new_final_21_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,595541,25000.000,21,27213.240,26900.000,26900.000,27213.240,0.988,0.988,1.000,69500.000,38000.000,-31500.000,Decrease in Limits,-6500.000,0.000,-25000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7112836,26900.000,21,29367.580,28944.400,28944.400,29367.580,0.986,0.986,1.000,26900.000,24900.000,-2000.000,Decrease in Limits,24900.000,24900.000,-2000.000,1892.400,378.480,27170.880,26779.354,391.526,415.018,27194.372,26802.507,391.865,747.396,27549.903,27549.903
2,7933465,1600.000,21,1745.610,1721.600,1721.600,1745.610,0.986,0.986,1.000,0.000,0.000,0.000,No Change in Limits,1600.000,1600.000,0.000,121.600,24.320,1745.920,1721.906,24.014,25.455,1747.361,1723.327,24.034,45.840,1769.167,1769.167
3,674584,84800.000,21,92546.230,91244.800,91244.800,92546.230,0.986,0.986,1.000,84800.000,107200.000,22400.000,Increase in Limits,107200.000,107200.000,22400.000,8147.200,1629.440,116976.640,115331.658,1644.982,1743.681,117075.339,115428.969,1646.370,3140.090,118569.058,118569.058
4,915474,17600.000,21,22532.170,3770.000,5420.000,6628.090,0.167,0.241,0.294,17600.000,0.000,-17600.000,Decrease in Limits,0.000,0.000,-17600.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [151]:
refresh_four_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  33765497.835
new_amount_transaction_by_end_rollover_date   35759339.992
new_amount_transaction_by_dpd_90_date         37425895.162
dtype: float64

In [152]:
refresh_four_21_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_four_21_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  1.002
new_amount_transaction_by_end_rollover_date   1.061
new_amount_transaction_by_dpd_90_date         1.110
dtype: float64

In [153]:
refresh_four_21_day_limits_and_loans['new_loan_amount'].sum()

33711800.0

##### 7 Day Loans

In [154]:
refresh_four_7_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-31') & (final_df['disbursed_on_date'] <= '2022-04-07') & (final_df['term_frequency'] == 7)]

refresh_four_7_day_loans = refresh_four_7_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_four_7_day_limits_and_loans = pd.merge(refresh_four_7_day_loans, refresh_four_7_day_limits, on = "store_number", how = 'left')

refresh_four_7_day_limits_and_loans['new_loan_amount_raw'] = refresh_four_7_day_limits_and_loans['principal_disbursed'] + refresh_four_7_day_limits_and_loans['limits_difference']

refresh_four_7_day_limits_and_loans['new_loan_amount'] = refresh_four_7_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_four_7_day_limits_and_loans['loan_difference'] = refresh_four_7_day_limits_and_loans['new_loan_amount'] - refresh_four_7_day_limits_and_loans['principal_disbursed']

refresh_four_7_day_limits_and_loans['facility_fee'] = refresh_four_7_day_limits_and_loans['new_loan_amount'] * 2.60/100
refresh_four_7_day_limits_and_loans['excise_duty'] = refresh_four_7_day_limits_and_loans['new_loan_amount'] * 0.52/100
refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_four_7_day_limits_and_loans['new_loan_amount'] + refresh_four_7_day_limits_and_loans['facility_fee'] + refresh_four_7_day_limits_and_loans['excise_duty']
#refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 103/100

refresh_four_7_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7215187,60000.000,7,62446.520,26100.000,62460.000,62460.000,0.418,1.000,1.000,143800.000,200000.000,56200.000,Increase in Limits,116200.000,116200.000,56200.000,3021.200,604.240,119825.440
1,7233376,4000.000,7,4476.010,0.000,0.000,4993.520,0.000,0.000,1.116,3200.000,4000.000,800.000,Increase in Limits,4800.000,4800.000,800.000,124.800,24.960,4949.760


In [155]:
refresh_four_7_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_four_7_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_four_7_day_limits_and_loans['rollover_amount'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_four_7_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_four_7_day_limits_and_loans['updated_rollover_amount'] = refresh_four_7_day_limits_and_loans['rollover_amount'] * 103/100
refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_7_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_four_7_day_limits_and_loans['updated_rollover_amount']

refresh_four_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_four_7_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_four_7_day_limits_and_loans['default_amount'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_four_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_four_7_day_limits_and_loans['updated_default_amount'] = refresh_four_7_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_four_7_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_four_7_day_limits_and_loans['updated_default_amount']

refresh_four_7_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_four_7_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_four_7_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_four_7_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_7_limit,new_final_7_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7215187,60000.000,7,62446.520,26100.000,62460.000,62460.000,0.418,1.000,1.000,143800.000,200000.000,56200.000,Increase in Limits,116200.000,116200.000,56200.000,3021.200,604.240,119825.440,50081.958,69743.482,71835.787,121917.744,121944.062,-26.318,NaN,NaN,NaN
1,7233376,4000.000,7,4476.010,0.000,0.000,4993.520,0.000,0.000,1.116,3200.000,4000.000,800.000,Increase in Limits,4800.000,4800.000,800.000,124.800,24.960,4949.760,0.000,4949.760,5098.253,5098.253,0.000,5098.253,9723.797,9723.797,10848.049
2,972636,22100.000,7,23242.670,0.000,23006.100,23242.670,0.000,0.990,1.000,NaN,NaN,NaN,NaN,NaN,0.000,-22100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,7087650,5500.000,7,5780.510,0.000,5725.500,5780.510,0.000,0.990,1.000,9200.000,11500.000,2300.000,Increase in Limits,7800.000,7800.000,2300.000,202.800,40.560,8043.360,0.000,8043.360,8284.661,8284.661,8205.820,78.841,150.371,8356.191,8356.191
4,7649161,19100.000,7,20494.070,0.000,19883.100,20494.070,0.000,0.970,1.000,36900.000,34200.000,-2700.000,Decrease in Limits,16400.000,16400.000,-2700.000,426.400,85.280,16911.680,0.000,16911.680,17419.030,17419.030,16899.734,519.297,990.445,17890.178,17890.178


In [156]:
refresh_four_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  57080216.928
new_amount_transaction_by_end_rollover_date   64708429.254
new_amount_transaction_by_dpd_90_date         63973901.068
dtype: float64

In [157]:
refresh_four_7_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_four_7_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.895
new_amount_transaction_by_end_rollover_date   1.015
new_amount_transaction_by_dpd_90_date         1.003
dtype: float64

In [158]:
refresh_four_7_day_limits_and_loans['new_loan_amount'].sum()

63765657.0

##### 1 Day Loans

In [159]:
refresh_four_1_day_loans = final_df[(final_df['disbursed_on_date'] >= '2022-03-31') & (final_df['disbursed_on_date'] <= '2022-04-07') & (final_df['term_frequency'] == 1)]

refresh_four_1_day_loans = refresh_four_1_day_loans[['store_number', 'principal_disbursed', 'term_frequency', 'total_expected_repayment','amount_trans_by_due_date', 'amount_transaction_by_end_rollover_date', 'amount_transaction_by_dpd_90_date', 'repayments_ratio_by_due_date', 'repayments_ratio_by_end_rollover_date', 'repayments_ratio_by_dpd_90_date']]

refresh_four_1_day_limits_and_loans = pd.merge(refresh_four_1_day_loans, refresh_four_1_day_limits, on = "store_number", how = 'left')

refresh_four_1_day_limits_and_loans['new_loan_amount_raw'] = refresh_four_1_day_limits_and_loans['principal_disbursed'] + refresh_four_1_day_limits_and_loans['limits_difference']

refresh_four_1_day_limits_and_loans['new_loan_amount'] = refresh_four_1_day_limits_and_loans['new_loan_amount_raw'].apply(lambda x: max(0, x))

refresh_four_1_day_limits_and_loans['loan_difference'] = refresh_four_1_day_limits_and_loans['new_loan_amount'] - refresh_four_1_day_limits_and_loans['principal_disbursed']

refresh_four_1_day_limits_and_loans['facility_fee'] = refresh_four_1_day_limits_and_loans['new_loan_amount'] * 0.40/100
refresh_four_1_day_limits_and_loans['excise_duty'] = refresh_four_1_day_limits_and_loans['new_loan_amount'] * 0.08/100
refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] = refresh_four_1_day_limits_and_loans['new_loan_amount'] + refresh_four_1_day_limits_and_loans['facility_fee'] + refresh_four_1_day_limits_and_loans['excise_duty']
#refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * 100.48/100

refresh_four_1_day_limits_and_loans.head(2)

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date
0,7629469,200.000,1,257.900,0.000,0.000,3.600,0.000,0.000,0.014,NaN,NaN,NaN,NaN,NaN,0.000,-200.000,0.000,0.000,0.000
1,7311274,400.000,1,404.600,401.600,401.600,404.600,0.993,0.993,1.000,1800.000,1200.000,-600.000,Decrease in Limits,-200.000,0.000,-400.000,0.000,0.000,0.000


In [160]:
refresh_four_1_day_limits_and_loans['new_amount_trans_by_due_date'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] * refresh_four_1_day_limits_and_loans['repayments_ratio_by_due_date']
refresh_four_1_day_limits_and_loans['rollover_amount'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_due_date'] - refresh_four_1_day_limits_and_loans['new_amount_trans_by_due_date']
refresh_four_1_day_limits_and_loans['updated_rollover_amount'] = refresh_four_1_day_limits_and_loans['rollover_amount'] * 100.48/100
refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] = refresh_four_1_day_limits_and_loans['new_amount_trans_by_due_date'] + refresh_four_1_day_limits_and_loans['updated_rollover_amount']

refresh_four_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] * refresh_four_1_day_limits_and_loans['repayments_ratio_by_end_rollover_date']
refresh_four_1_day_limits_and_loans['default_amount'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_end_rollover_date'] - refresh_four_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date']
refresh_four_1_day_limits_and_loans['updated_default_amount'] = refresh_four_1_day_limits_and_loans.apply(lambda x: daily_default_charge(x), axis = 1)
refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] = refresh_four_1_day_limits_and_loans['new_amount_transaction_by_end_rollover_date'] + refresh_four_1_day_limits_and_loans['updated_default_amount']

refresh_four_1_day_limits_and_loans['new_amount_transaction_by_dpd_90_date'] = refresh_four_1_day_limits_and_loans['updated_new_loan_amount_by_dpd_90_date'] * refresh_four_1_day_limits_and_loans['repayments_ratio_by_dpd_90_date']

refresh_four_1_day_limits_and_loans.head()

,store_number,principal_disbursed,term_frequency,total_expected_repayment,amount_trans_by_due_date,amount_transaction_by_end_rollover_date,amount_transaction_by_dpd_90_date,repayments_ratio_by_due_date,repayments_ratio_by_end_rollover_date,repayments_ratio_by_dpd_90_date,final_1_limit,new_final_1_limit,limits_difference,limits_difference_bands,new_loan_amount_raw,new_loan_amount,loan_difference,facility_fee,excise_duty,updated_new_loan_amount_by_due_date,new_amount_trans_by_due_date,rollover_amount,updated_rollover_amount,updated_new_loan_amount_by_end_rollover_date,new_amount_transaction_by_end_rollover_date,default_amount,updated_default_amount,updated_new_loan_amount_by_dpd_90_date,new_amount_transaction_by_dpd_90_date
0,7629469,200.000,1,257.900,0.000,0.000,3.600,0.000,0.000,0.014,NaN,NaN,NaN,NaN,NaN,0.000,-200.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,7311274,400.000,1,404.600,401.600,401.600,404.600,0.993,0.993,1.000,1800.000,1200.000,-600.000,Decrease in Limits,-200.000,0.000,-400.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,474197,10000.000,1,10115.000,10040.000,10040.000,10115.000,0.993,0.993,1.000,92900.000,70200.000,-22700.000,Decrease in Limits,-12700.000,0.000,-10000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,7801918,400.000,1,401.600,401.600,401.600,401.600,1.000,1.000,1.000,2700.000,16400.000,13700.000,Increase in Limits,14100.000,14100.000,13700.000,56.400,11.280,14167.680,14167.680,0.000,0.000,14167.680,14167.680,0.000,0.000,14167.680,14167.680
4,660899,1100.000,1,1104.400,1104.400,1104.400,1104.400,1.000,1.000,1.000,NaN,NaN,NaN,NaN,NaN,0.000,-1100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [161]:
refresh_four_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum()

new_amount_trans_by_due_date                  3466878.898
new_amount_transaction_by_end_rollover_date   3751763.550
new_amount_transaction_by_dpd_90_date         3843855.076
dtype: float64

In [162]:
refresh_four_1_day_limits_and_loans[['new_amount_trans_by_due_date', 'new_amount_transaction_by_end_rollover_date', 'new_amount_transaction_by_dpd_90_date']].sum() / refresh_four_1_day_limits_and_loans['new_loan_amount'].sum()

new_amount_trans_by_due_date                  0.909
new_amount_transaction_by_end_rollover_date   0.984
new_amount_transaction_by_dpd_90_date         1.008
dtype: float64

In [163]:
refresh_four_1_day_limits_and_loans['new_loan_amount'].sum()

3812115.0